# قم ببناء نموذج انحدار: ابدأ باستخدام R وTidymodels لنماذج الانحدار


## مقدمة إلى الانحدار - الدرس الأول

#### وضع الأمور في سياقها

✅ هناك العديد من أنواع طرق الانحدار، واختيار الطريقة المناسبة يعتمد على الإجابة التي تبحث عنها. إذا كنت تريد التنبؤ بالطول المحتمل لشخص بناءً على عمره، ستستخدم `الانحدار الخطي`، لأنك تبحث عن **قيمة رقمية**. أما إذا كنت مهتمًا بمعرفة ما إذا كان نوع معين من المأكولات يجب اعتباره نباتيًا أم لا، فأنت تبحث عن **تصنيف فئة**، وبالتالي ستستخدم `الانحدار اللوجستي`. ستتعلم المزيد عن الانحدار اللوجستي لاحقًا. فكر قليلاً في بعض الأسئلة التي يمكنك طرحها على البيانات، وأي من هذه الطرق سيكون أكثر ملاءمة.

في هذا القسم، ستعمل مع [مجموعة بيانات صغيرة عن مرض السكري](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html). تخيل أنك تريد اختبار علاج لمرضى السكري. قد تساعدك نماذج التعلم الآلي في تحديد المرضى الذين قد يستجيبون بشكل أفضل للعلاج، بناءً على مجموعات من المتغيرات. حتى نموذج انحدار بسيط جدًا، عند تصوره، قد يظهر معلومات عن المتغيرات التي يمكن أن تساعدك في تنظيم تجاربك السريرية النظرية.

مع ذلك، دعونا نبدأ في هذه المهمة!

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>عمل فني بواسطة @allison_horst</figcaption>

<!--![عمل فني بواسطة \@allison_horst](../../../../../../translated_images/encouRage.e75d5fe0367fb9136b78104baf4e2032a7622bc42a2bc34c0ad36c294eeb83f5.ar.jpg)<br>عمل فني بواسطة @allison_horst-->


## 1. تحميل مجموعة الأدوات الخاصة بنا

لهذه المهمة، سنحتاج إلى الحزم التالية:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) هو [مجموعة من حزم R](https://www.tidyverse.org/packages) مصممة لجعل علم البيانات أسرع وأسهل وأكثر متعة!

-   `tidymodels`: إطار عمل [tidymodels](https://www.tidymodels.org/) هو [مجموعة من الحزم](https://www.tidymodels.org/packages/) للنمذجة وتعلم الآلة.

يمكنك تثبيتها باستخدام الأمر التالي:

`install.packages(c("tidyverse", "tidymodels"))`

السكربت أدناه يتحقق مما إذا كانت الحزم المطلوبة لإكمال هذا الوحدة مثبتة لديك، ويقوم بتثبيتها إذا كانت مفقودة.


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



الآن، دعونا نقوم بتحميل هذه الحزم الرائعة وجعلها متاحة في جلسة R الحالية لدينا. (هذا للتوضيح فقط، `pacman::p_load()` قد قام بذلك بالفعل نيابةً عنك)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. مجموعة بيانات مرض السكري

في هذا التمرين، سنعرض مهاراتنا في الانحدار من خلال إجراء توقعات باستخدام مجموعة بيانات مرض السكري. تحتوي [مجموعة بيانات مرض السكري](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) على `442 عينة` من البيانات المتعلقة بمرض السكري، مع 10 متغيرات تنبؤية تشمل `العمر`، `الجنس`، `مؤشر كتلة الجسم`، `متوسط ضغط الدم`، و`ستة قياسات لمصل الدم` بالإضافة إلى متغير النتيجة `y`: وهو قياس كمي لتقدم المرض بعد سنة واحدة من نقطة البداية.

|عدد الملاحظات|442|
|----------------------|:---|
|عدد المتغيرات التنبؤية|أول 10 أعمدة هي تنبؤية رقمية|
|النتيجة/الهدف|العمود 11 هو قياس كمي لتقدم المرض بعد سنة واحدة من نقطة البداية|
|معلومات المتغيرات التنبؤية|- العمر بالسنوات
||- الجنس
||- مؤشر كتلة الجسم (bmi)
||- متوسط ضغط الدم (bp)
||- s1 tc، إجمالي الكوليسترول في المصل
||- s2 ldl، البروتينات الدهنية منخفضة الكثافة
||- s3 hdl، البروتينات الدهنية عالية الكثافة
||- s4 tch، إجمالي الكوليسترول / HDL
||- s5 ltg، ربما لوغاريتم مستوى الدهون الثلاثية في المصل
||- s6 glu، مستوى السكر في الدم|

> 🎓 تذكر، هذا تعلم تحت الإشراف، ونحتاج إلى هدف مسمى 'y'.

قبل أن تتمكن من التعامل مع البيانات باستخدام R، تحتاج إلى استيراد البيانات إلى ذاكرة R، أو إنشاء اتصال بالبيانات يمكن لـ R استخدامه للوصول إلى البيانات عن بُعد.

> توفر حزمة [readr](https://readr.tidyverse.org/)، وهي جزء من Tidyverse، طريقة سريعة وسهلة لقراءة البيانات المستطيلة إلى R.

الآن، دعونا نقوم بتحميل مجموعة بيانات مرض السكري من رابط المصدر التالي: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

سنقوم أيضًا بإجراء فحص سريع على بياناتنا باستخدام `glimpse()` وعرض أول 5 صفوف باستخدام `slice()`.

قبل المضي قدمًا، دعونا نقدم شيئًا ستواجهه كثيرًا في كود R 🥁🥁: مشغل الأنابيب `%>%`

مشغل الأنابيب (`%>%`) يقوم بتنفيذ العمليات بتسلسل منطقي عن طريق تمرير كائن إلى وظيفة أو تعبير استدعاء. يمكنك التفكير في مشغل الأنابيب كأنه يقول "ثم" في الكود الخاص بك.


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` يظهر لنا أن هذه البيانات تحتوي على 442 صفًا و11 عمودًا، وكل الأعمدة من نوع البيانات `double`.

<br>

> glimpse() و slice() هما دالتان في [`dplyr`](https://dplyr.tidyverse.org/). Dplyr، وهو جزء من Tidyverse، يمثل قواعد لتنظيم البيانات ويوفر مجموعة متناسقة من الأفعال التي تساعدك في حل التحديات الأكثر شيوعًا في معالجة البيانات.

<br>

الآن بعد أن حصلنا على البيانات، دعونا نركز على خاصية واحدة (`bmi`) كهدف لهذا التمرين. هذا يتطلب منا اختيار الأعمدة المطلوبة. كيف يمكننا القيام بذلك؟

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) تتيح لنا *اختيار* (وأحيانًا إعادة تسمية) الأعمدة في إطار البيانات.


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. بيانات التدريب والاختبار

من الممارسات الشائعة في التعلم الموجّه *تقسيم* البيانات إلى مجموعتين؛ مجموعة (عادةً أكبر) لتدريب النموذج، ومجموعة أصغر "محجوزة" لمعرفة أداء النموذج.

الآن بعد أن أصبحت البيانات جاهزة، يمكننا معرفة ما إذا كانت الآلة يمكن أن تساعد في تحديد تقسيم منطقي بين الأرقام في هذه المجموعة. يمكننا استخدام حزمة [rsample](https://tidymodels.github.io/rsample/)، وهي جزء من إطار عمل Tidymodels، لإنشاء كائن يحتوي على معلومات حول *كيفية* تقسيم البيانات، ثم استخدام وظيفتين إضافيتين من rsample لاستخراج مجموعتي التدريب والاختبار التي تم إنشاؤهما:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. تدريب نموذج الانحدار الخطي باستخدام Tidymodels

الآن نحن جاهزون لتدريب النموذج!

في Tidymodels، يتم تحديد النماذج باستخدام `parsnip()` من خلال تحديد ثلاثة مفاهيم:

-   **نوع النموذج** يميز بين النماذج مثل الانحدار الخطي، الانحدار اللوجستي، نماذج الأشجار القرار، وما إلى ذلك.

-   **وضع النموذج** يشمل خيارات شائعة مثل الانحدار والتصنيف؛ بعض أنواع النماذج تدعم أيًا من هذه الخيارات بينما البعض الآخر لديه وضع واحد فقط.

-   **محرك النموذج** هو الأداة الحسابية التي سيتم استخدامها لتدريب النموذج. غالبًا ما تكون هذه الأدوات حزم R، مثل **`"lm"`** أو **`"ranger"`**

يتم جمع هذه المعلومات الخاصة بالنمذجة في مواصفات النموذج، لذا دعونا نبني واحدة!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

بعد أن يتم *تحديد* النموذج، يمكن `تقديره` أو `تدريبه` باستخدام وظيفة [`fit()`](https://parsnip.tidymodels.org/reference/fit.html)، وعادةً ما يتم ذلك باستخدام صيغة وبيانات معينة.

`y ~ .` تعني أننا سنقوم بتقدير `y` كالقيمة المتوقعة/الهدف، والتي يتم تفسيرها بواسطة جميع المتغيرات التوضيحية/الخصائص أي `.` (في هذه الحالة، لدينا متغير توضيحي واحد فقط: `bmi`).


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

من المخرجات التي يقدمها النموذج، يمكننا رؤية المعاملات التي تم تعلمها أثناء التدريب. هذه المعاملات تمثل معاملات خط الانحدار الأفضل الذي يعطينا أقل خطأ إجمالي بين المتغير الفعلي والمتغير المتوقع.
<br>

## 5. إجراء التنبؤات على مجموعة الاختبار

الآن بعد أن قمنا بتدريب النموذج، يمكننا استخدامه للتنبؤ بتطور المرض y لمجموعة بيانات الاختبار باستخدام [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). سيتم استخدام هذا لرسم الخط الفاصل بين مجموعات البيانات.


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

رائع! 💃🕺 لقد قمنا بتدريب نموذج واستخدمناه لإجراء التنبؤات!

عند إجراء التنبؤات، تتبع tidymodels دائمًا قاعدة إنتاج tibble/إطار بيانات يحتوي على أسماء أعمدة موحدة. هذا يجعل من السهل دمج البيانات الأصلية مع التنبؤات في صيغة قابلة للاستخدام للعمليات اللاحقة مثل الرسم البياني.

`dplyr::bind_cols()` يربط أعمدة إطارات البيانات المتعددة بكفاءة.


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. عرض نتائج النمذجة

حان الوقت لرؤية هذا بشكل مرئي 📈. سنقوم بإنشاء مخطط مبعثر لجميع قيم `y` و `bmi` في مجموعة الاختبار، ثم نستخدم التوقعات لرسم خط في المكان الأنسب بين مجموعات بيانات النموذج.

لدى R عدة أنظمة لإنشاء الرسوم البيانية، ولكن `ggplot2` هو واحد من أكثرها أناقة وتنوعًا. يتيح لك هذا النظام تكوين الرسوم البيانية عن طريق **دمج المكونات المستقلة**.


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ فكر قليلاً فيما يحدث هنا. هناك خط مستقيم يمر عبر العديد من النقاط الصغيرة للبيانات، لكن ما الذي يفعله بالضبط؟ هل يمكنك أن ترى كيف يمكن استخدام هذا الخط للتنبؤ بمكان نقطة بيانات جديدة وغير مرئية بالنسبة لمحور y في الرسم البياني؟ حاول أن تصف بالكلمات الاستخدام العملي لهذا النموذج.

تهانينا، لقد قمت ببناء أول نموذج انحدار خطي، وأنشأت توقعًا باستخدامه، وعرضته في رسم بياني!



---

**إخلاء المسؤولية**:  
تمت ترجمة هذا المستند باستخدام خدمة الترجمة الآلية [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية هو المصدر الموثوق. للحصول على معلومات حساسة أو هامة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة تنشأ عن استخدام هذه الترجمة.
